In [2]:
import pandas as pd
import os
import urllib.request
import socket
import random

socket.setdefaulttimeout(20)

CSV_PATH = "./en.openfoodfacts.org.products.csv"
IMAGES_PATH = "./data/images/"
LABEL_PATH = "./data/labels.csv"
ROOT_URI = "https://images.openfoodfacts.org/images/products/"

In [3]:
N_train_public = 50000
N_test_public = 5000
N_train_private, N_test_private = N_train_public, N_test_public

filtered_categories = ["Beverages", "Sweet snacks", "Dairies", "Cereals and potatoes", "Meats", "Fermented foods", "Fermented milk products", 
                    "Groceries", "Meals", "Cereals and their products", "Cheeses", "Sauces", "Spreads", "Confectioneries", "Prepared meats", 
                    "Frozen foods", "Breakfasts", "Desserts", "Canned foods", "Seafood", "Cocoa and its products", "Fats", "Condiments", 
                    "Fishes", "Breads", "Yogurts", "Cakes", "Biscuits", "Pastas", "Legumes"]
filtered_categories = [s.lower() for s in filtered_categories]

CLASS_TO_INDEX = {filtered_categories[i]:i for i in range(len(filtered_categories))}
INDEX_TO_CLASS = {i:filtered_categories[i] for i in range(len(filtered_categories))}

In [4]:
def idx_to_str(idx):
    tmp_idx = idx
    zeros = 7
    while tmp_idx // 10 > 0:
        tmp_idx = tmp_idx // 10
        zeros -= 1
    return "0"*zeros + str(idx)

def filter_categories(categories_str):
    s = categories_str.lower().split(',')
    out = ""
    for label in CLASS_TO_INDEX:
        if label in s:
            out += str(CLASS_TO_INDEX[label]) + ";"
            continue
    if len(out) == 0: return out
    return out[:-1] # Remove last coma

In [5]:
skip_fact = 5

In [6]:
df = pd.read_csv(CSV_PATH, sep="\t").dropna(how='any', subset=["categories_en", "image_url"])[["categories_en", "image_url"]]

output_df_public = pd.DataFrame(columns=["split", "uri", "labels"])
output_df_private = pd.DataFrame(columns=["split", "uri", "labels"])

C:\Users\babal\Miniconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0,8,14,20,21,22,23,24,28,29,30,32,33,42,43,44,49,63) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
df.head()

,categories_en,image_url
7,"Groceries,Condiments,Sauces,Mustards",https://images.openfoodfacts.org/images/produc...
16,"Marzipan,Almond paste",https://images.openfoodfacts.org/images/produc...
17,"Meals,Prepared vegetables,Grated carrots,Seaso...",https://images.openfoodfacts.org/images/produc...
26,"Pies,Sweet pies,Coconut pies",https://images.openfoodfacts.org/images/produc...
36,"Plant-based foods and beverages,Plant-based fo...",https://images.openfoodfacts.org/images/produc...


In [8]:
N = N_test_private + N_test_public + N_train_public + N_train_private
idx_test_private, idx_train_private, idx_test_public, idx_train_public = 0, 0, 0, 0


iterator = df.iterrows()

# BongoSort complexity?
while N_test_private + N_test_public + N_train_public + N_train_private > 0:
    for _ in range(skip_fact):
        idx, row = next(iterator)
    out = filter_categories(row["categories_en"])
    while out == "":
        idx, row = next(iterator)
        out = filter_categories(row["categories_en"])

    r = random.random()
    if r < N_test_private/N:
        output_df_private.loc[idx_test_private+idx_train_private] = {"split": "test", "uri": row["image_url"][len(ROOT_URI):], "labels":str(out)}
        idx_test_private += 1
        N_test_private -= 1
    elif r < (N_test_private + N_test_public)/N:
        output_df_public.loc[idx_test_public+idx_train_public] = {"split": "test", "uri": row["image_url"][len(ROOT_URI):], "labels":str(out)}
        idx_test_public += 1
        N_test_public -= 1
    elif r < (N_test_private + N_test_public + N_train_public)/N:
        output_df_public.loc[idx_train_public+idx_test_public] = {"split": "train", "uri": row["image_url"][len(ROOT_URI):], "labels":str(out)}
        idx_train_public += 1
        N_train_public -=1
    else:
        output_df_private.loc[idx_train_private+idx_test_private] = {"split": "train", "uri": row["image_url"][len(ROOT_URI):], "labels":str(out)}
        idx_train_private += 1
        N_train_private -= 1
    N -= 1
print(idx_test_private)
print(idx_test_public)
print(idx_train_private)
print(idx_train_public)
assert idx_test_public == idx_test_private
assert idx_train_private == idx_train_public

output_df_private.to_csv("./data_location_private.csv", )
output_df_public.to_csv("./data_location_public.csv")

5000
5000
50000
50000
